In [1]:
import numpy as np
import pandas as pd
import json 

import bokeh
from bokeh.models.widgets import Panel, Tabs
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.sampledata.commits import data
from bokeh.transform import jitter

bokeh.io.output_notebook()

Loading BokehJS ...

In [12]:
df = pd.read_csv('../Lung_Paper_Images/Quantification/Kidney/20200703_Stitched_Kidney_Cell_Counts.csv', comment='#')
df.head()

,Date,Size Threshold,Gaussian Size,Truncation,Intensity Threshold,Minimum Size,Virus,Animal,Replicate,Count,Cells Quantified,Brightness List,Applied Threshold,Image Multiplication Factor,Minimum Pixel Value,Maximum Pixel Value,Area Threshold,Total Area
0,2020-07-03,0.6,5,2,1000,10,AAV5,1,1,22,[ 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 ...,"{'1': 0.06830289860909684, '2': 0.028398599968...",0.455161,0.513324,-0.223618,0.300823,0.018103,2.264285e+07
1,2020-07-03,0.6,5,2,1000,10,AAV5,1,2,32,[ 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 ...,"{'1': 0.03824426711823111, '2': 0.030321369432...",0.249780,0.516100,-0.082208,0.304475,0.016811,2.233620e+07
2,2020-07-03,0.6,5,2,1000,10,AAV5,2,1,21,[ 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 ...,"{'1': 0.034836969839451684, '2': 0.19173473715...",0.219212,0.525355,-0.120577,0.494698,0.017996,2.248486e+07
3,2020-07-03,0.6,5,2,1000,10,AAV5,2,2,23,[ 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 ...,"{'1': 0.03466153044213417, '2': 0.087344368793...",0.403291,0.538369,-0.217963,0.358672,0.017880,2.418042e+07
4,2020-07-03,0.6,5,2,1000,10,AAV5,3,1,7,[1 2 3 4 5 6 7],"{'1': 0.13308138710686004, '2': 0.032385963101...",0.243497,0.388340,-0.086250,0.305901,0.017610,2.050852e+07


In [13]:
# Using Determine the transduction as a function of tissue area
df['Cells Transduced per mm2'] = df['Count'] / df['Total Area'] * 1000000

# Get the median brightness from the brightness dictionary
for inds in np.arange(0, len(df)):     
    # Load the brightness dictionary, replacing the single quotation marks in the keys with double quotation marks for json compliance. 
    brightness_dict = json.loads(df.loc[inds, 'Brightness List'].replace("\'", "\""))
    
    # Determine the median of each brightness dictionary and store it in the dataframe
    df.loc[inds, 'Brightness Median'] = np.median(list(brightness_dict.values()))

#Sort the rows 
df = df.sort_values(by=['Virus', 'Animal', 'Replicate'])

# Rename the specific rows for plotting purposes. 
df['Virus'] = df['Virus'].replace('CAPA4','AAV.CAP-A4')

In [14]:
#Group the data by the specific virus and animals
grouped = df.groupby(['Virus','Animal'])

# Make two new dataframes that track cell transduction and median cell brightness respectively
df_counts = grouped['Cells Transduced per mm2'].mean().reset_index()
df_brightness = grouped['Brightness Median'].mean().reset_index()

In [15]:
#Initialize an output file to save an html of the generated plot
#output_file("../Lung_Paper_Images/Quantification/Lung/lung_transduction_per_sq_mm.html")

# Store the data for plotting
source = ColumnDataSource(df_counts)

# Create the catagories to plot from
catagories = list(df['Virus'].unique())

# Intitialize a figure
p = figure(plot_width=300, plot_height=300, x_range=catagories, y_axis_label = 'Cells Transduced per mm\u00b2')

# Plot the data as a scatter plot
p.circle(x=jitter('Virus', width=0.6, range=p.x_range), y='Cells Transduced per mm2', source=source, alpha=1)

# Simply modify plot characteristics 
p.plot_height = 350
p.plot_width = 350

p.xaxis.major_label_text_font_size = '12pt'
p.yaxis.major_label_text_font_size = '12pt'

p.xaxis.axis_label_text_font_size = '18pt'
p.yaxis.axis_label_text_font_size = '18pt'
show(p)

In [16]:
#Initialize an output file to save an html of the generated plot
#output_file("../Lung_Paper_Images/Quantification/Lung/lung_median_brightness.html")

# Store the data for plotting
source = ColumnDataSource(df_brightness)

# Create the catagories to plot from
catagories = list(df['Virus'].unique())

# Intitialize a figure
p = figure(plot_width=300, plot_height=300, x_range=catagories, y_axis_label = 'Median Cell Brightness', title="Lung")

# Plot the data as a scatter plot
p.circle(x=jitter('Virus', width=0.6, range=p.x_range), y='Brightness Median', source=source, alpha=1)

p.plot_height = 350
p.plot_width = 350

p.xaxis.major_label_text_font_size = '12pt'
p.yaxis.major_label_text_font_size = '12pt'

p.xaxis.axis_label_text_font_size = '18pt'
p.yaxis.axis_label_text_font_size = '18pt'

show(p)